In [2]:
## Import necessary libraries
import os
import random
import cv2
import numpy as np
import random
import math
from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
from google.colab.patches import cv2_imshow
%matplotlib inline


In [3]:
# Download Data
!gdown 1A0Rin_YMmWkExjI99vfLYvU_dy-9gFTT
!unzip "/content/Part2_data.zip" -d "/content/"
# Load Matches
data = loadmat('/content/Part2_data/tracks.mat')

Downloading...
From: https://drive.google.com/uc?id=1A0Rin_YMmWkExjI99vfLYvU_dy-9gFTT
To: /content/Part2_data.zip
100% 5.44M/5.44M [00:00<00:00, 134MB/s]
Archive:  /content/Part2_data.zip
   creating: /content/Part2_data/
   creating: /content/Part2_data/images/
  inflating: /content/Part2_data/images/hotel.seq0.png  
  inflating: /content/Part2_data/images/hotel.seq1.png  
  inflating: /content/Part2_data/images/hotel.seq10.png  
  inflating: /content/Part2_data/images/hotel.seq11.png  
  inflating: /content/Part2_data/images/hotel.seq12.png  
  inflating: /content/Part2_data/images/hotel.seq13.png  
  inflating: /content/Part2_data/images/hotel.seq14.png  
  inflating: /content/Part2_data/images/hotel.seq15.png  
  inflating: /content/Part2_data/images/hotel.seq16.png  
  inflating: /content/Part2_data/images/hotel.seq17.png  
  inflating: /content/Part2_data/images/hotel.seq18.png  
  inflating: /content/Part2_data/images/hotel.seq19.png  
  inflating: /content/Part2_data/images/hot

In [4]:
# Remove the nan value
# Make the pints zero mean,  create function and feed them to affine SFM

from numpy.linalg.linalg import isnan
import scipy


track_x = data['track_x']
track_y = data['track_y']

def nan_mean(track_x, track_y):
  tmp_x= np.transpose(track_x)
  tmp_y= np.transpose(track_y)
  #Remove nan
  tmp_x =(tmp_x[:, ~np.isnan(tmp_x).any(axis=0)])
  #Zero mean
  for i in range(tmp_x.shape[0]):
    tmp_x[i]= tmp_x[i]-np.mean(tmp_x[i])

  # Remove nan
  tmp_y =(tmp_y[:, ~np.isnan(tmp_y).any(axis=0)])
  #Zero mean
  for j in range(tmp_y.shape[0]):
    tmp_y[j]= tmp_y[j]-np.mean(tmp_y[j])
  
  track_x= np.transpose(tmp_x)
  track_y= np.transpose(tmp_y)

  return track_x, track_y

# def validate_cov_matrix(M):
#     M = (M + M.T) * 0.5
#     k = 0
#     I = np.eye(M.shape[0])
#     while True:
#         try:
#             _ = np.linalg.cholesky(M)
#             break
#         except np.linalg.LinAlgError:
#             k += 1
#             w, v = np.linalg.eig(M)
#             min_eig = v.min()
#             M += (-min_eig * k * k + np.spacing(min_eig)) * I
#     return M


def affineSFM(x, y):
  '''
  Function: Affine structure from motion algorithm
  % Normalize x, y to zero mean
  % Create measurement matrix
  D = [xn' ; yn'];
  % Decompose and enforce rank 3
  % Apply orthographic constraints
  '''

  # Construct matrix D
  D = np.block([[x[:, i], y[:, i]] for i in range(x.shape[1])])
  D=D.reshape((102, -1))
  # D = np.block([[x[:, i], y[:, i]] for i in range(y.shape[1])]).reshape((102, -1))
  # Compute SVD
  U,W,V= np.linalg.svd(D)
 
  W= np.diag(W)
  W_3= W[:3, :3]
  U_3= U[:, :3]
  V_3= V[:3, :] # Might have to transpose here, check back

  # Create motion and shape matrices
  # print(U_3)
  A = U_3 @ np.sqrt(W_3)
  S = np.sqrt(W_3) @ V_3

  # Calculate matrices Q and R to calculate L 
  R=[]
  
  for i in range(0, A.shape[0], 2):
    a1= A[i]
    a2= A[i+1]
    # Create 3 equations for each image and stack them to form matrix Q
    eq_1 = np.dot(np.expand_dims(a1, axis=1),np.expand_dims(a1, axis=0))
    eq_1=eq_1.reshape((1,9))
    eq_2 = np.dot(np.expand_dims(a2, axis=1),np.expand_dims(a2, axis=0))
    eq_2= eq_2.reshape((1,9))
    eq_3 = np.dot(np.expand_dims(a1, axis=1),np.expand_dims(a2, axis=0))
    eq_3=eq_3.reshape((1,9))
    
    if i!=0:
      Q = np.concatenate((Q, eq_1), axis=0)
    else:
      Q = eq_1
      
    Q = np.concatenate((Q, eq_2), axis=0)
    Q = np.concatenate((Q, eq_3), axis=0)
    R += [1,1,0]
  
  R = (np.array(R)).reshape(-1,1)
  Q = np.array(Q)
  L= np.dot(np.linalg.pinv(Q),R)
  L=L.reshape((3,3))
  # L= validate_cov_matrix(L)
  # print(L)
  C= np.linalg.cholesky(L)
  A_new = np.dot(A,C)
  S_new = np.dot(np.linalg.inv(C), S)

  return A_new, S_new

def plot_motion(A_new):
  for i in range(0, A.shape[0], 2):
    kf= np.cross(A[i], A[i+1])
    normalized = kf / np.linalg.norm(kf)
    normalized = np.expand_dims(normalized, axis=0)
    if i==0:
      camera_pos = normalized
    else:
      camera_pos =  np.concatenate((camera_pos, normalized), axis=0)
  
  posX = camera_pos[:, 0]
  posY = camera_pos[:, 1]
  posZ = camera_pos[:, 2]
  return posX, posY, posZ

In [5]:
import plotly.graph_objects as go

x,y= nan_mean(track_x, track_y)
A, S= affineSFM(x,y)
#PLot 3d figure
fig = go.Figure(data=[go.Scatter3d(x=S[0], y=S[1], z=S[2], mode='markers',marker=dict(size=3))])
fig.show()

# plot camera motion
x1, y1, z1= plot_motion(A)
plt.plot(x1)
plt.title('x_plot')
plt.plot(y1)
plt.title('y_plot')
plt.plot(z1)
plt.title('z_plot')